# Hamiltonian Exponentiation - 4th Place Zeeshan Ahmed, India
Below is code and methodology from the fourth place solution to the Spring 2022 Classiq Coding Competition.

### Note
This code was originally in separate files. The code from these files has been pasted here.

#### run.py

In [ ]:
from lietrotter import Lie
from qdrift_trotter import Qdrift
from suzuki import Suzuki
from constructor import hamiltonian_circuit_error
from parser import store_dict_as_json, export_circuit
from circuit_optimizer import Optimizer

import numpy as np
from typing import List
from matplotlib import pyplot as plt


def get_min_reps(constructor, reps_s=1, reps_e=5, threshold=0.1):
    """
    Iterates over reps for the given constructor until the threshold is
    achieved.

    Args:
        constructor: Constructor for getting circuit
        reps_s: starting value of reps
        reps_2: ending value of reps
        threshold: error threshold

    """
    errs = []
    depths = []

    pauli_op = constructor.pauli_op

    min_depth = np.inf
    min_circuit = None
    max_reps = reps_e + 1
    optimizer = Optimizer()

    for reps in range(reps_s, reps_e + 1):
        constructor.re_init(reps)
        print(f"----------> Running {constructor.method} on {reps} reps.")
        constructor.get_circuit()
        dec_circuit = constructor.decompose_circuit(optimizer)

        error = hamiltonian_circuit_error(dec_circuit, pauli_op)
        depth = dec_circuit.depth()

        errs.append(error)
        depths.append(depth)

        if min_depth > depth:
            min_depth = depth
            min_circuit = dec_circuit
        if error <= threshold:
            max_reps = reps + 1
            break

    return {
        "min_circuit": min_circuit,
        "error": errs,
        "depth": depths,
        "reps": list(range(reps_s, max_reps)),
    }


def plot_error_vs_depth(list_dicts: List[dict], threshold=0.1, savefig=True):
    """
    Plots multiple error vs depth comparison for different methods.

    Args:
        list_dicts: List of dicts which contains `error`, `depth` and `label` as it's
                    keys.
    """

    print(list_dicts)
    plt.figure(figsize=(20, 12))
    for dict_ in list_dicts:
        plt.plot(
            dict_["depth"],
            dict_["error"],
            linestyle="--",
            marker="o",
            label=dict_["label"],
        )
        for ind, rep in enumerate(dict_["reps"]):
            plt.annotate(f"{rep}", (dict_["depth"][ind], dict_["error"][ind]))

    plt.axhline(y=threshold, label="error threshold")

    plt.xlabel("Depth")
    plt.ylabel("Error")
    plt.title("Depth vs Error for different trotter methods")
    plt.grid()
    plt.legend()

    if savefig:
        all_labels = "--".join([x["label"] for x in list_dicts])
        plt.savefig(f"plots/{all_labels}.png")
    plt.show()


def run_methods(constructors: List, hamiltonian: str, reps_s=1, reps_e=5):
    """
    run all the methods and store the best circuit for each method based
    on the name of the method and other parameters.

    Args:
        - constructors: list of constructors
        - reps_s: starting rep number
        - reps_e: ending rep number
    """

    list_dicts = []

    for constructor in constructors:
        constructor.load_hamiltonian(hamiltonian)
        data = get_min_reps(constructor, reps_s, reps_e)
        depth = data["min_circuit"].depth()
        export_circuit(
            data["min_circuit"],
            f"circuits/{constructor.hamiltonian}_{constructor.method}_{depth}.qasm",
        )
        list_dicts.append(
            {
                **data,
                "label": f"{constructor.hamiltonian}_{constructor.method}_{reps_s}_to_{reps_e}",
            }
        )
    plot_error_vs_depth(list_dicts)


def main():
    hamiltonian = "LiH"
    optimizer = Optimizer()

    constructors = [
        Lie(optimizer=optimizer),
        Qdrift(optimizer=optimizer),
        Suzuki(optimizer=optimizer),
    ]
    run_methods(constructors, hamiltonian, 1, 5)
    return


if __name__ == "__main__":
    main()

#### lietrotter.py

In [ ]:
from qiskit.synthesis import LieTrotter
from constructor import Constructor, hamiltonian_circuit_error
import numpy as np


class Lie(Constructor):
    def __init__(self, _reps: int = 1, optimizer=None):
        super().__init__("LIE", optimizer)
        assert _reps > 0, "Incorrect number of reps provided"
        self.reps = _reps
        self.synthesizer = LieTrotter(reps=_reps)
        assert self.synthesizer is not None, "Error constructing the circuit."

    def re_init(self, _reps):
        """Re-initialize synthesizer
        Re-initializes the synthesizer with new number of reps.

        Args:
            _reps: Number of times QDRIFT must be repeated in the circuit.
        Raises:
            AssertionError: Incorrect rep count.
            AssertionError: QDrift could not construct circuit.
        """
        assert _reps > 0, "Incorrect number of reps provided"
        self.synthesizer = LieTrotter(reps=_reps)
        assert self.synthesizer is not None, "Error constructing the circuit."


def loop(a, b, print_circuit, hamiltonian):
    constructor = Lie()
    constructor.load_hamiltonian(hamiltonian)
    min_err = np.inf
    for reps in range(a, b):
        constructor.re_init(reps)
        circ = constructor.get_circuit()
        pauli_op = constructor.pauli_op
        dec_circuit = constructor.decompose_circuit()
        if print_circuit:
            print(circ.decompose())
        error = hamiltonian_circuit_error(dec_circuit, pauli_op)
        if error < min_err:
            min_err = error

        depth = dec_circuit.depth()
        print(f"reps:{reps} error:{error}, depth:{depth}")

    print(f"{min_err}")


def main():
    loop(1, 2, True, "H2")


if __name__ == "__main__":
    main()


#### qdrift_trotter.py

In [ ]:
from qiskit.synthesis import QDrift
from constructor import Constructor, hamiltonian_circuit_error

import numpy as np


class Qdrift(Constructor):
    def __init__(self, _reps: int = 1, optimizer=None):
        super().__init__("QDRIFT", optimizer)
        assert _reps > 0, "Incorrect number of reps provided"
        self.reps = _reps
        self.synthesizer = QDrift(reps=_reps)

    def re_init(self, _reps):
        """Re-initialize synthesizer
        Re-initializes the synthesizer with new number of reps.

        Args:
            _reps: Number of times QDRIFT must be repeated in the circuit.
        Raises:
            AssertionError: Incorrect rep count.
            AssertionError: QDrift could not construct circuit.
        """
        assert _reps > 0, "Incorrect number of reps provided"
        self.synthesizer = QDrift(reps=_reps)
        assert self.synthesizer is not None, "Error constructing the circuit."


def loop(a, b, print_circuit, hamiltonian):
    constructor = Qdrift()
    constructor.load_hamiltonian(hamiltonian)
    min_err = np.inf
    for reps in range(a, b):
        constructor.re_init(reps)
        constructor.get_circuit()
        pauli_op = constructor.pauli_op
        dec_circuit = constructor.decompose_circuit()
        if print_circuit:
            print(dec_circuit)
        error = hamiltonian_circuit_error(dec_circuit, pauli_op)
        if error < min_err:
            min_err = error

        depth = dec_circuit.depth()
        print(f"reps:{reps} error:{error}, depth:{depth}")

    print(f"{min_err}")


def main():
    loop(1, 4, False, "H2")


if __name__ == "__main__":
    main()


#### suzuki.py

In [ ]:
from qiskit.synthesis import SuzukiTrotter
from constructor import Constructor


class Suzuki(Constructor):
    def __init__(self, order: int = 1, optimizer=None):
        super().__init__("SUZUKI", optimizer)
        assert order > 0, "Incorrect order."
        self.order = order

    def re_init(self, _reps):
        """Re-initialize synthesizer
        Re-initializes the synthesizer with new number of reps.

        Args:
            _reps: Number of times QDRIFT must be repeated in the circuit.
        Raises:
            AssertionError: Incorrect rep count.
            AssertionError: QDrift could not construct circuit.
        """
        assert _reps > 0, "Incorrect number of reps provided"
        self.synthesizer = SuzukiTrotter(reps=_reps)
        assert self.synthesizer is not None, "Error constructing the circuit."


#### constructor.py

In [ ]:
from typing import List, Union, Optional
from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from parser import get_pauli_sum_op
from qiskit.opflow import PauliSumOp
from qiskit import Aer, transpile
from qiskit.extensions.hamiltonian_gate import HamiltonianGate

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

import numpy as np
import cmath


def global_phase(a: np.ndarray):
    # get the phase of the first non-zero value in the matrix
    for row in a:
        for x in row:
            if np.abs(x) > 1e-3:
                theta = cmath.phase(x)
                # phase is e^{i\theta}
                phase = np.exp(0 + 1j * theta)
                # print(x, theta, phase)
                return phase

    return 1 + 0j


def get_circuit_unitary(circuit, decimals: int = 8, backend=None):
    if backend is None:
        backend = Aer.get_backend("aer_simulator")
    unitary = None
    # have to make copy to avoid changes in the original circuit due to save_unitary.
    new_circuit = circuit.copy()
    new_circuit.save_unitary()
    transpiled = transpile(new_circuit, backend)

    result = backend.run(transpiled).result()
    unitary = result.get_unitary(transpiled, decimals=decimals)
    unitary = unitary.data / global_phase(unitary.data)
    return unitary


def get_hamiltonian_unitary(pauli_op):
    hamiltonian = HamiltonianGate(pauli_op, time=1)
    unitary = hamiltonian.to_matrix()
    unitary = unitary / global_phase(unitary)
    return unitary


def hamiltonian_circuit_error(circuit: QuantumCircuit, hamiltonian: PauliSumOp):

    circ_unitary = get_circuit_unitary(circuit)
    hamiltonian_unitary = get_hamiltonian_unitary(hamiltonian)

    diff_mat = circ_unitary - hamiltonian_unitary
    return np.linalg.norm(diff_mat, 2)


def decompose_circuit(
    circuit: QuantumCircuit, gate_set: Optional[List] = None
) -> Union[QuantumCircuit, List[QuantumCircuit]]:
    if gate_set is None:
        gate_set = ["u3", "cx"]

    transpile_pass = Unroller(gate_set)
    manager = PassManager(transpile_pass)
    return manager.run(circuit)


class Constructor:
    def __init__(self, method: str, optimizer=None):
        self.method = method
        self.hamiltonian = None
        self.circuit = None
        self.synthesizer = None
        self.optimizer = optimizer

    def load_hamiltonian(self, hamiltonian: str, optimizer=None):
        """Load given Hamiltonian

        Loads the given Hamiltonian from the folder `hamiltonian/`.
        Hamiltonian when read from a file can further be optimized

        Args:
            - hamiltonian: name of the hamiltonian file
            - optimizer: callable that will take in the pauli_op and optimize it
        """
        self.hamiltonian = hamiltonian
        pauli_op: PauliSumOp = get_pauli_sum_op(hamiltonian)
        if optimizer is not None:
            pauli_op = optimizer(pauli_op)

        self.pauli_op = pauli_op

    def re_init(self):
        """
        Re-initialize the synthesizer for the construction of circuit.
        """
        raise NotImplemented("Accessing superclass is not allowed")

    def get_circuit(self) -> QuantumCircuit:
        """Get Hamiltonian circuit.

        Get the higher level circuit for the Hamiltonian.
        Note that the circuit must be converted to fundamental gates for
        measuring the gate depth.

        Returns:
            Quantum Circuit for the loaded hamiltonian.
        """
        evo_gate = PauliEvolutionGate(self.pauli_op, 1.0, synthesis=self.synthesizer)

        num_qubits = evo_gate.num_qubits
        self.num_qubits = num_qubits
        circ = QuantumCircuit(num_qubits)  # reset circuit everytime to avoid pile-up
        circ.append(evo_gate, list(range(num_qubits)))

        self.circuit = circ
        return circ

    def decompose_circuit(self, optimizer=None) -> QuantumCircuit:
        assert self.circuit is not None, "Circuit not prepared yet."

        dec_circuit = decompose_circuit(self.circuit)
        if type(dec_circuit) is list:
            raise NotImplemented("decompose_circuit returns a list.")

        if optimizer is not None:
            dec_circuit = optimizer(dec_circuit)
        elif self.optimizer is not None:
            dec_circuit = self.optimizer(dec_circuit)

        self.dec_circuit = dec_circuit
        assert type(dec_circuit) is QuantumCircuit
        return dec_circuit


#### parser.py

In [ ]:
from typing import List
from qiskit import QuantumCircuit
from qiskit.opflow import PauliSumOp, PauliOp, SummedOp
import json


def export_circuit(circuit: QuantumCircuit, file_name: str):
    if type(circuit) is not QuantumCircuit:
        raise NotImplemented("exports only circuits.")

    qasm_string = circuit.qasm()
    assert qasm_string is not None, "Could not generate qasm string"

    with open(file_name, "w") as f:
        f.write(qasm_string)

def import_circuit(file_name: str):
    return QuantumCircuit.from_qasm_file(file_name)


def get_string_from_file(file_name: str) -> str:
    string = ""
    with open(file_name, "r") as file:
        string = file.read()

    return string


def parse_to_terms(string: str) -> List:
    lines = list(filter(lambda x: len(x) > 0, string.splitlines()))
    pauli_pairs = []
    for line in lines:
        coeff, axis = line.split("*")
        coeff = coeff.replace(" ", "")

        axis = axis.strip()
        pauli_pairs.append((axis, coeff))

    return pauli_pairs


def get_pauli_sum_op(hamiltonian: str) -> PauliSumOp:
    string = get_string_from_file(f"hamiltonians/{hamiltonian}")
    pauli_pairs = parse_to_terms(string)
    return PauliSumOp.from_list(pauli_pairs)


def get_pauli_list(pauli_sum_op: PauliSumOp) -> List:
    pauli_op = pauli_sum_op.to_pauli_op()

    if type(pauli_op) is PauliOp:
        return [(pauli_op.primitive, pauli_op.coeff)]
    else:
        assert type(pauli_op) is SummedOp
        sum_op = list(pauli_op)
        return [(str(x.primitive), x.coeff) for x in sum_op]


def store_dict_as_json(req_dict: dict, file_name: str):
    with open(file_name, "w") as fp:
        json.dump(req_dict, fp)


def load_json(file_name: str):
    with open(file_name, "r") as fp:
        data = json.load(fp)

    return data


def main():
    hamiltonian = "H2"
    pauli_op = get_pauli_sum_op(hamiltonian)
    print(pauli_op, type(pauli_op))
    pauli_list = get_pauli_list(pauli_op)
    print(pauli_list)


if __name__ == "__main__":
    main()


#### circuit_optimizer.py

In [ ]:
from qiskit import QuantumCircuit
from lietrotter import Lie
from constructor import hamiltonian_circuit_error
from parser import export_circuit

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import (
    Optimize1qGatesSimpleCommutation,
    CommutativeCancellation,
    CXCancellation,
    CommutationAnalysis,
)


class Optimizer:
    def __init__(self, basis_gates=None):
        if basis_gates is None:
            basis_gates = ["u3", "cx"]
        self.basis_gates = basis_gates

        simple_commutation_pass = Optimize1qGatesSimpleCommutation(basis=basis_gates)
        cancel_cx_pass = CXCancellation()
        commutative_cancellation_pass = CommutativeCancellation(basis_gates=basis_gates)
        commutative_analysis_pass = CommutationAnalysis()

        self.passes = [
            simple_commutation_pass,
            cancel_cx_pass,
            commutative_analysis_pass,
            commutative_cancellation_pass,
        ]
        self.manager = PassManager(self.passes)

    @staticmethod
    def _pass_call_back(original_depth, dict_):
        print(f"{dict_['pass_']} completed in {dict_['time']}")
        print(f"original depth: {original_depth} new depth: {dict_['dag'].depth()}")

    @staticmethod
    def _end_call_back(**kwargs):
        print(f'new:{kwargs["new_depth"]} old:{kwargs["original_depth"]}')

    def transipile_optimize(self, circuit: QuantumCircuit) -> QuantumCircuit:
        circ = circuit.copy()
        original_depth = circ.depth()

        def call_back(**kwargs):
            self._pass_call_back(original_depth, kwargs)

        transpiled_circ = self.manager.run(circ, callback=call_back)

        assert (
            type(transpiled_circ) is QuantumCircuit
        ), "transpile returned list, not supported currently."
        new_depth = transpiled_circ.depth()
        self._end_call_back(new_depth=new_depth, original_depth=original_depth)

        return transpiled_circ

    def __call__(self, circuit: QuantumCircuit) -> QuantumCircuit:
        return self.transipile_optimize(circuit)


def main():
    hamiltonian = "H2"
    constructor = Lie(1)
    constructor.load_hamiltonian(hamiltonian)
    constructor.get_circuit()
    pauli_op = constructor.pauli_op
    dec_circuit = constructor.decompose_circuit()

    optimizer = Optimizer()
    optim_circ = optimizer.transipile_optimize(dec_circuit)
    new_error = hamiltonian_circuit_error(optim_circ, pauli_op)
    original_error = hamiltonian_circuit_error(dec_circuit, pauli_op)

    print(f"original error:{original_error}, new error:{new_error}")
    export_circuit(
        optim_circ,
        f"circuits/{constructor.hamiltonian}_{constructor.method}_{optim_circ.depth()}.qasm",
    )


if __name__ == "__main__":
    main()


#### check_qasm.py

In [ ]:
import parser
import constructor

def main():
    file_name = "qasm/final.qasm"
    circ = parser.import_circuit(file_name)
    print(f"depth:{circ.depth()}")
    pauli = parser.get_pauli_sum_op("LiH")
    print(f"error: {constructor.hamiltonian_circuit_error(circ, pauli)}")


if __name__ == "__main__":
    main()



#### grouping_optimizer.py

In [ ]:
from pickletools import decimalnl_long
from requests import delete
from qiskit.opflow import PauliSumOp
import copy
from parser import get_pauli_list
from hamiltonian_optimizer import BaseHamiltonianOptimizer
import random
from pprint import pprint
from circuit_optimizer import Optimizer as Circ_optimizer
from optimized_order import order, coeffs
from order import order as ordered_grouping

class bcolors: 
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"


class PairWiseOptimizer(BaseHamiltonianOptimizer):
    def __init__(self, iter):
        self.num_deletes = 8
        self.iter = iter

    def randomize(self, d):
        keys = list(d.keys())
        random.shuffle(keys)

        coeff_map = dict()
        for key in keys:
            coeff_map[key] = d[key]
        return coeff_map

    def delete_single_group(self, d):
        pass

    def group_by_coeff(self, pauli_list):
        coeff_map = dict()

        for pauli, coeff in pauli_list:
            if coeff in coeff_map:
                coeff_map[coeff].append(pauli)
            else:
                coeff_map[coeff] = [pauli]

        new_list = []
        # coeff_map = self.randomize(coeff_map)
        for coeff, p_list in coeff_map.items():
            new_list.append((coeff, p_list))

        return new_list

    def group_by_u(self, strings):
        nonI_map = dict()
        pauli = ["I", "X", "Y", "Z"]

        for string in strings:
            u_string = string
            for nonI in pauli[1:]:
                u_string = u_string.replace(nonI, "U")

            if u_string in nonI_map:
                nonI_map[u_string].append(string)
            else:
                nonI_map[u_string] = []
                nonI_map[u_string].append(string)

        matched = set()
        new_list = []

        for u_string, u_set in nonI_map.items():
            if u_string in matched:
                continue

            u_list = list(u_set)
            new_list.extend(u_list)

            flip_u_string = (
                u_string.replace("U", "X").replace("I", "U").replace("X", "I")
            )

            if flip_u_string not in nonI_map:
                continue
            flip_set = nonI_map[flip_u_string]
            flip_u_list = sorted(list(flip_set))
            new_list.extend(flip_u_list)

            matched.add(u_string)
            matched.add(flip_u_string)

        assert len(new_list) == len(strings)
        return new_list

    def delete_coffient_group(self, pauli_list):
        """
        delete terms with low coefficints
        """
        new_list = copy.deepcopy(pauli_list)
        for i in range(self.num_deletes):
            deleted_list = []
            min_coff = min([x[1] for x in new_list])
            for term in pauli_list:
                if term[1] > min_coff:
                    deleted_list.append(term)
            new_list = copy.deepcopy(deleted_list)

        # print("newlist len: ", len(new_list))
        return new_list

    def delete_term(self, pauli_list):
        """
        delete a term given its index
        """

        pauli_list.pop(self.iter)
        return pauli_list

    def delete_group(self, pauli_list):
        """
        delete a group given its coeff
        """
        new_list = []
        for string, coeff in pauli_list:
            if coeff != coeffs[self.iter]:
                new_list.append((string, coeff))
        return new_list

    def intra_group_swipe(self, pauli_list):
        """
        Bring reflexives closer within a given group
        """
        new_list = []
        for string, coeff in pauli_list:
            reflected_string = string[5:] + string[:5]
            if reflected_string != string and (reflected_string, coeff) in pauli_list:
                new_list.append((string, coeff))
                new_list.append((reflected_string, coeff))
                pauli_list.remove((reflected_string, coeff))
            else:
                new_list.append((string, coeff))
        return new_list

    def optimize(self, pauli_op: PauliSumOp) -> PauliSumOp:
        """
        Method for optimizing the Pauli operator.

        Args:
            - pauli_op: Pauli operator to be optimized
        Returns:
            - optimized: Pauli operator
        """
        # pauli_list = get_pauli_list(pauli_op)
        # new_pair_list = self.group_by_coeff(pauli_list)

        # new_list = []
        # for coeff, p_list in new_pair_list:
        #     p_list = self.group_by_u(p_list)
        #     for p in p_list:
        #         new_list.append((p, coeff))

        # new_list = self.intra_group_swipe(new_list)
        # pprint(new_list)
        # exit(0)
        new_list = ordered_grouping
        # new_list = self.delete_group(new_list)

        return PauliSumOp.from_list(new_list)

    def __call__(self, pauli_op: PauliSumOp) -> PauliSumOp:
        return self.optimize(pauli_op)


def main(iter):
    from constructor import hamiltonian_circuit_error
    from lietrotter import Lie

    hamiltonian = "LiH"
    optimizer = PairWiseOptimizer(iter)
    circ_optimizer = Circ_optimizer()

    constructor = Lie(optimizer=circ_optimizer)

    constructor.load_hamiltonian(hamiltonian=hamiltonian)
    original_pauli_op = constructor.pauli_op
    constructor.load_hamiltonian(hamiltonian, optimizer=optimizer)

    pauli_op = constructor.pauli_op
    constructor.get_circuit()
    dec_circuit = constructor.decompose_circuit()
    error = hamiltonian_circuit_error(dec_circuit, original_pauli_op)
    depth = dec_circuit.depth()
    print(
        f"{bcolors.WARNING}Optimized error:{error}, depth:{depth} with len(H): {len(original_pauli_op)}{bcolors.ENDC}"
    )

    if error < 0.1 and depth < 2000:
        dec_circuit.qasm(filename=f"QASM--{iter}--{depth}--{error}.qasm")

    return (error, depth)

    # constructor.load_hamiltonian(hamiltonian=hamiltonian)
    # constructor.get_circuit()
    # pauli_op = constructor.pauli_op
    # dec_circuit = constructor.decompose_circuit()
    # error = hamiltonian_circuit_error(dec_circuit, original_pauli_op)
    # depth = dec_circuit.depth()
    # print(
    #     f"{bcolors.WARNING}Unoptimized error:{error}, depth:{depth} with len(H): {len(original_pauli_op)}{bcolors.ENDC}"
    # )


if __name__ == "__main__":
    # del_error = []
    # for i in range(len(coeffs)):
    #     del_error.append((main(i), coeffs[i]))
    # pprint(del_error)
    main(1)

#### aqc.py

In [ ]:
import numpy as np

from qiskit.transpiler.synthesis.aqc import (
    make_cnot_network,
    AQC,
    CNOTUnitCircuit,
    DefaultCNOTUnitObjective,
)
from qiskit.algorithms.optimizers import L_BFGS_B


def get_approximate_circuit(unitary: np.ndarray, **kwargs):
    num_qubits = int(round(np.log2(unitary.shape[0])))
    default_params = {
        "network_layout": "sequ",
        "connectivity_type": "full",
        "depth": 50,
    }

    params = {**default_params, **kwargs}

    # Generate a network made of CNOT units
    cnots = make_cnot_network(num_qubits=num_qubits, **params)

    optimizer = L_BFGS_B(maxiter=10, iprint=99)
    aqc = AQC(optimizer)
    approximating_objective = DefaultCNOTUnitObjective(
        num_qubits=num_qubits, cnots=cnots
    )

    approximate_circuit = CNOTUnitCircuit(num_qubits=num_qubits, cnots=cnots)
    aqc.compile_unitary(
        target_matrix=unitary,
        approximate_circuit=approximate_circuit,
        approximating_objective=approximating_objective,
    )
    return approximate_circuit


def main():
    from lietrotter import Lie
    from constructor import get_hamiltonian_unitary, hamiltonian_circuit_error

    hamiltonian = "LiH"
    constructor = Lie()
    constructor.load_hamiltonian(hamiltonian)
    unitary = get_hamiltonian_unitary(constructor.pauli_op)
    approximate_circuit = get_approximate_circuit(unitary)

    print(approximate_circuit.depth())
    print(hamiltonian_circuit_error(approximate_circuit, constructor.pauli_op))


if __name__ == "__main__":
    main()


#### randompermutation.py

In [ ]:
from qiskit.opflow import PauliSumOp
from parser import get_pauli_list, get_pauli_sum_op
from hamiltonian_optimizer import BaseHamiltonianOptimizer
from pprint import pprint as print
from circuit_optimizer import Optimizer as Circ_optimizer

from random import shuffle, randint


class RandomShuffle(BaseHamiltonianOptimizer):
    def __init__(self, shuffle_count=10):
        self.shuffle_count = shuffle_count

    def optimize(self, pauli_op: PauliSumOp) -> PauliSumOp:
        """
        Method for optimizing the Pauli operator.

        Args:
            - pauli_op: Pauli operator to be optimized
        Returns:
            - optimized: Pauli operator
        """
        pauli_list = get_pauli_list(pauli_op)  # [(string, coeff)]
        shuffle_count = self.shuffle_count

        position = randint(0, len(pauli_list) - (shuffle_count + 1))

        shuffled = pauli_list[position : position + shuffle_count]
        shuffle(shuffled)

        for x in range(shuffle_count):
            pauli_list[x + position] = shuffled[x]

        return PauliSumOp.from_list(pauli_list)

    def __call__(self, pauli_op: PauliSumOp) -> PauliSumOp:
        return self.optimize(pauli_op)


def main():
    from constructor import hamiltonian_circuit_error
    from lietrotter import Lie

    hamiltonian = "LiH"
    circ_optimizer = Circ_optimizer()

    # constructor = Lie(optimizer=circ_optimizer)
    constructor = Lie()
    for ind in range(200, 240, 10):
        optimizer = RandomShuffle(ind)
        constructor.load_hamiltonian(hamiltonian, optimizer=optimizer)
        pauli_op = constructor.pauli_op
        constructor.get_circuit()
        dec_circuit = constructor.decompose_circuit()
        error = hamiltonian_circuit_error(dec_circuit, pauli_op)
        depth = dec_circuit.depth()
        print(f"{ind} optimized error:{error}, depth:{depth}")

    constructor.load_hamiltonian(hamiltonian=hamiltonian)
    constructor.get_circuit()
    # pauli_op = constructor.pauli_op
    dec_circuit = constructor.decompose_circuit()
    error = hamiltonian_circuit_error(dec_circuit, pauli_op)
    depth = dec_circuit.depth()
    print(f"unoptimized error:{error}, depth:{depth}")


if __name__ == "__main__":
    main()


#### pairwise_optimizer.py

In [ ]:
from qiskit.opflow import PauliSumOp
from parser import get_pauli_list, get_pauli_sum_op
from hamiltonian_optimizer import BaseHamiltonianOptimizer
from pprint import pprint as print
from circuit_optimizer import Optimizer as Circ_optimizer


class PairWiseOptimizer(BaseHamiltonianOptimizer):
    def __init__(self):
        pass

    def group(self, strings):
        nonI_map = dict()

        pauli = ["I", "X", "Y", "Z"]
        for string in strings:
            u_string = string
            for nonI in pauli[1:]:
                u_string = u_string.replace(nonI, "U")

            if u_string in nonI_map:
                nonI_map[u_string].append(string)
            else:
                nonI_map[u_string] = []
                nonI_map[u_string].append(string)

        matched = set()
        new_list = []

        for u_string, u_set in nonI_map.items():
            if u_string in matched:
                continue

            u_list = list(u_set)
            new_list.extend(u_list)

            flip_u_string = (
                u_string.replace("U", "X").replace("I", "U").replace("X", "I")
            )

            if flip_u_string not in nonI_map:
                continue
            flip_set = nonI_map[flip_u_string]
            flip_u_list = sorted(list(flip_set))
            new_list.extend(flip_u_list)

            matched.add(u_string)
            matched.add(flip_u_string)

        assert len(new_list) == len(strings)
        return new_list

    def optimize(self, pauli_op: PauliSumOp) -> PauliSumOp:
        """
        Method for optimizing the Pauli operator.

        Args:
            - pauli_op: Pauli operator to be optimized
        Returns:
            - optimized: Pauli operator
        """
        pauli_list = get_pauli_list(pauli_op)
        pauli_strings = [x[0] for x in pauli_list]
        coeff_map = {x: y for (x, y) in pauli_list}
        new_list = self.group(pauli_strings)

        new_pauli_list = []
        for x in new_list:
            new_pauli_list.append((x, coeff_map[x]))

        return PauliSumOp.from_list(new_pauli_list)

    def __call__(self, pauli_op: PauliSumOp) -> PauliSumOp:
        return self.optimize(pauli_op)


def main():
    from constructor import hamiltonian_circuit_error
    from lietrotter import Lie

    hamiltonian = "LiH"
    optimizer = PairWiseOptimizer()
    circ_optimizer = Circ_optimizer()

    constructor = Lie(optimizer=circ_optimizer)
    constructor.load_hamiltonian(hamiltonian, optimizer=optimizer)

    pauli_op = constructor.pauli_op
    constructor.get_circuit()

    dec_circuit = constructor.decompose_circuit()
    error = hamiltonian_circuit_error(dec_circuit, pauli_op)
    depth = dec_circuit.depth()
    print(f"optimized error:{error}, depth:{depth}")

    constructor.load_hamiltonian(hamiltonian=hamiltonian)
    constructor.get_circuit()
    pauli_op = constructor.pauli_op
    dec_circuit = constructor.decompose_circuit()
    error = hamiltonian_circuit_error(dec_circuit, pauli_op)
    depth = dec_circuit.depth()
    print(f"unoptimized error:{error}, depth:{depth}")


if __name__ == "__main__":
    main()


#### lexico_optimizer.py

In [ ]:
from qiskit.opflow import PauliSumOp
from parser import get_pauli_list
from hamiltonian_optimizer import BaseHamiltonianOptimizer


class LexicoOptimizer(BaseHamiltonianOptimizer):
    def __init__(self):
        pass

    def optimize(self, pauli_op: PauliSumOp) -> PauliSumOp:
        """
        Method for optimizing the Pauli operator.

        Args:
            - pauli_op: Pauli operator to be optimized
        Returns:
            - optimized: Pauli operator
        """
        pauli_list = get_pauli_list(pauli_op)
        pauli_list.sort()

        return PauliSumOp.from_list(pauli_list)

    def __call__(self, pauli_op: PauliSumOp) -> PauliSumOp:
        return self.optimize(pauli_op)


def main():
    from lietrotter import Lie
    from constructor import hamiltonian_circuit_error

    hamiltonian = "LiH"
    optimizer = LexicoOptimizer()
    constructor = Lie()
    constructor.load_hamiltonian(hamiltonian=hamiltonian, optimizer=optimizer)

    pauli_op = constructor.pauli_op
    constructor.get_circuit()
    dec_circuit = constructor.decompose_circuit()
    error = hamiltonian_circuit_error(dec_circuit, pauli_op)
    depth = dec_circuit.depth()
    print(f"optimized error:{error}, depth:{depth}")

    constructor.load_hamiltonian(hamiltonian=hamiltonian)
    constructor.get_circuit()
    pauli_op = constructor.pauli_op
    dec_circuit = constructor.decompose_circuit()
    error = hamiltonian_circuit_error(dec_circuit, pauli_op)
    depth = dec_circuit.depth()
    print(f"unoptimized error:{error}, depth:{depth}")


if __name__ == "__main__":
    main()


#### hamiltonian_optimizer.py

In [ ]:
from qiskit.opflow import PauliSumOp


class BaseHamiltonianOptimizer:
    def __init__(self):
        pass

    def __call__(self, pauli_op: PauliSumOp) -> PauliSumOp:
        """
        Method for optimizing the Pauli operator.

        Args:
            - pauli_op: Pauli operator to be optimized
        Returns:
            - optimized: Pauli operator
        """
        raise NotImplemented("Do not use the base class for creating instances.")


#### order.py

In [ ]:
order = [
    ("IIIYYIIIYY", 0.006264656830204855),
    # ("IIIXXIIIYY", 0.003164656830204855),
    # ("IIIYYIIIXX", 0.003164656830204855),
    ("IIIXXIIIXX", 0.006264656830204855),
    ("YZZZYIIIYY", -0.008373361424264817),
    ("IIIYYYZZZY", -0.008373361424264817),
    ("XZZZXIIIYY", -0.008373361424264817),
    ("IIIYYXZZZX", -0.008373361424264817),
    ("YZZZYIIIXX", -0.008373361424264817),
    ("IIIXXYZZZY", -0.008373361424264817),
    ("XZZZXIIIXX", -0.008373361424264817),
    ("IIIXXXZZZX", -0.008373361424264817),
    # ("YZZYIIIIYY", 0.00211113766859809),
    # ("IIIYYYZZYI", 0.00211113766859809),
    # ("XZZXIIIIYY", 0.00211113766859809),
    # ("IIIYYXZZXI", 0.00211113766859809),
    # ("YZZYIIIIXX", 0.00211113766859809),
    # ("IIIXXYZZYI", 0.00211113766859809),
    # ("XZZXIIIIXX", 0.00211113766859809),
    # ("IIIXXXZZXI", 0.00211113766859809),
    ("IIIIIIIIYY", -0.00491756976241806),
    ("IIIIIIIIXX", -0.00491756976241806),
    ("ZIIIIIIIYY", 0.010540187409026488),
    ("IIIYYZIIII", 0.010540187409026488),
    ("ZIIIIIIIXX", 0.010540187409026488),
    ("IIIXXZIIII", 0.010540187409026488),
    ("IZIIIIIIYY", -0.0011822832324725804),
    ("IIIYYIZIII", -0.0011822832324725804),
    ("IZIIIIIIXX", -0.0011822832324725804),
    ("IIIXXIZIII", -0.0011822832324725804),
    ("IIZIIIIIYY", -0.0011822832324725804),
    ("IIIYYIIZII", -0.0011822832324725804),
    ("IIZIIIIIXX", -0.0011822832324725804),
    ("IIIXXIIZII", -0.0011822832324725804),
    # ("IIIZIIIIYY", -0.00154067008970742),
    # ("IIIYYIIIZI", -0.00154067008970742),
    # ("IIIZIIIIXX", -0.00154067008970742),
    # ("IIIXXIIIZI", -0.00154067008970742),
    ("IIIIZIIIYY", 0.011733623912074194),
    ("IIIYYIIIIZ", 0.011733623912074194),
    ("IIIIZIIIXX", 0.011733623912074194),
    ("IIIXXIIIIZ", 0.011733623912074194),
    ("IIIIIZIIYY", 0.0027757462269049522),
    ("ZIIYYIIIII", 0.0027757462269049522),
    ("IIIIIZIIXX", 0.0027757462269049522),
    ("ZIIXXIIIII", 0.0027757462269049522),
    ("IIIIIIZIYY", 0.0036202487558837123),
    ("IZIYYIIIII", 0.0036202487558837123),
    ("IIIIIIZIXX", 0.0036202487558837123),
    ("IZIXXIIIII", 0.0036202487558837123),
    ("IIIIIIIZYY", 0.0036202487558837123),
    ("IIZYYIIIII", 0.0036202487558837123),
    ("IIIIIIIZXX", 0.0036202487558837123),
    ("IIZXXIIIII", 0.0036202487558837123),
    ("IIYZYIIYZY", 0.007496760849734561),
    ("IIXZXIIYZY", 0.007496760849734561),
    ("IIYZYIIXZX", 0.005496760849734561),
    # ("IIXZXIIXZX", 0.005996760849734561),
    # ("IYZZYIYZZY", 0.005996760849734561),
    ("IXZZXIYZZY", 0.007496760849734561),
    ("IYZZYIXZZX", 0.005496760849734561),
    ("IXZZXIXZZX", 0.007496760849734561),
    ("IIYYIIIYZY", 0.005802531988356293),
    ("IIYZYIIYYI", 0.004802531988356293),
    ("IIXXIIIYZY", 0.005802531988356293),
    ("IIYZYIIXXI", 0.004802531988356293),
    ("IIYYIIIXZX", 0.005802531988356293),
    ("IIXZXIIYYI", 0.004802531988356293),
    ("IIXXIIIXZX", 0.005802531988356293),
    ("IIXZXIIXXI", 0.004802531988356293),
    ("IYZYIIYZZY", 0.005802531988356293),
    ("IYZZYIYZYI", 0.004802531988356293),
    ("IXZXIIYZZY", 0.004802531988356293),
    ("IYZZYIXZXI", 0.004802531988356293),
    ("IYZYIIXZZX", 0.004802531988356293),
    ("IXZZXIYZYI", 0.004802531988356293),
    ("IXZXIIXZZX", 0.004802531988356293),
    ("IXZZXIXZXI", 0.004802531988356293),
    # ("YZYIIIIYZY", -0.004879740484191497),
    # ("XZXIIIIYZY", -0.004879740484191497),
    # ("YZYIIIIXZX", -0.004879740484191497),
    # ("XZXIIIIXZX", -0.004879740484191497),
    # ("YYIIIIYZZY", -0.004879740484191497),
    # ("XXIIIIYZZY", -0.004879740484191497),
    # ("YYIIIIXZZX", -0.004879740484191497),
    # ("XXIIIIXZZX", -0.004879740484191497),
    ("YZZZYYZZZY", 0.0297383271773138),
    ("XZZZXYZZZY", 0.0297383271773138),
    ("YZZZYXZZZX", 0.0297383271773138),
    ("XZZZXXZZZX", 0.0297383271773138),
    ("YZZYIYZZZY", -0.0077644411821215335),
    # ("YZZZYYZZYI", -0.0077644411821215335),
    ("XZZXIYZZZY", -0.0077644411821215335),
    ("YZZZYXZZXI", -0.0077644411821215335),
    ("YZZYIXZZZX", -0.0077644411821215335),
    ("XZZZXYZZYI", -0.0077644411821215335),
    # ("XZZXIXZZZX", -0.0077644411821215335),
    ("XZZZXXZZXI", -0.0077644411821215335),
    ("IIIIIYZZZY", -0.005949019975734247),
    ("IIIIIXZZZX", -0.005949019975734247),
    ("ZIIIIYZZZY", -0.0351167704024114),
    ("YZZZYZIIII", -0.0351167704024114),
    ("ZIIIIXZZZX", -0.0351167704024114),
    ("XZZZXZIIII", -0.0351167704024114),
    # ("IZIIIYZZZY", 0.0027298828353264134),
    # ("YZZZYIZIII", 0.0027298828353264134),
    # ("IZIIIXZZZX", 0.0027298828353264134),
    # ("XZZZXIZIII", 0.0027298828353264134),
    # ("IIZIIYZZZY", 0.0027298828353264134),
    # ("YZZZYIIZII", 0.0027298828353264134),
    # ("IIZIIXZZZX", 0.0027298828353264134),
    # ("XZZZXIIZII", 0.0027298828353264134),
    # ("IIIZIYZZZY", 0.0023679368995844726),
    # ("YZZZYIIIZI", 0.0023679368995844726),
    # ("IIIZIXZZZX", 0.0023679368995844726),
    # ("XZZZXIIIZI", 0.0023679368995844726),
    ("IIIIZYZZZY", -0.03305872858775587),
    ("YZZZYIIIIZ", -0.03305872858775587),
    ("IIIIZXZZZX", -0.03305872858775587),
    ("XZZZXIIIIZ", -0.03305872858775587),
    ("IIIIIYIZZY", -0.0038498576488650843),
    ("YIZZYIIIII", -0.0038498576488650843),
    # ("IIIIIXIZZX", -0.0039498576488650843),
    # ("XIZZXIIIII", -0.0021498576488650843),
    # ("IIIIIYZIZY", -0.0021498576488650843),
    # ("YZIZYIIIII", -0.0021498576488650843),
    # ("IIIIIXZIZX", -0.0021498576488650843),
    ("XZIZXIIIII", -0.0038498576488650843),
    ("IIIIIYZZIY", 0.004479074568182561),
    ("YZZIYIIIII", 0.004479074568182561),
    ("IIIIIXZZIX", 0.004479074568182561),
    ("XZZIXIIIII", 0.004479074568182561),
    ("IIYYIIIYYI", 0.010328819322301622),
    ("IIXXIIIYYI", 0.010328819322301622),
    ("IIYYIIIXXI", 0.010328819322301622),
    ("IIXXIIIXXI", 0.010328819322301622),
    ("IYZYIIYZYI", 0.010328819322301622),
    ("IXZXIIYZYI", 0.010328819322301622),
    ("IYZYIIXZXI", 0.010328819322301622),
    ("IXZXIIXZXI", 0.010328819322301622),
    # ("YZYIIIIYYI", -0.003466391848475337),
    # ("IIYYIYZYII", -0.003466391848475337),
    # ("XZXIIIIYYI", -0.003466391848475337),
    # ("IIYYIXZXII", -0.003466391848475337),
    # ("YZYIIIIXXI", -0.003466391848475337),
    # ("IIXXIYZYII", -0.003466391848475337),
    ("XZXIIIIXXI", -0.005566391848475337),
    ("IIXXIXZXII", -0.005566391848475337),
    ("YYIIIIYZYI", -0.005566391848475337),
    ("IYZYIYYIII", -0.005566391848475337),
    # ("XXIIIIYZYI", -0.003466391848475337),
    # ("IYZYIXXIII", -0.003466391848475337),
    # ("YYIIIIXZXI", -0.003466391848475337),
    # ("IXZXIYYIII", -0.003466391848475337),
    # ("XXIIIIXZXI", -0.003466391848475337),
    # ("IXZXIXXIII", -0.003466391848475337),
    ("YZZYIYZZYI", 0.006575744899182541),
    ("XZZXIYZZYI", 0.006575744899182541),
    ("YZZYIXZZXI", 0.006575744899182541),
    ("XZZXIXZZXI", 0.006575744899182541),
    ("IIIIIYZZYI", 0.023557442395837284),
    ("IIIIIXZZXI", 0.023557442395837284),
    ("ZIIIIYZZYI", 0.010889407716094479),
    ("YZZYIZIIII", 0.010889407716094479),
    ("ZIIIIXZZXI", 0.010889407716094479),
    ("XZZXIZIIII", 0.010889407716094479),
    # ("IZIIIYZZYI", -0.0003518893528389501),
    # ("YZZYIIZIII", -0.0003518893528389501),
    # ("IZIIIXZZXI", -0.0003518893528389501),
    # ("XZZXIIZIII", -0.0003518893528389501),
    # ("IIZIIYZZYI", -0.0003518893528389501),
    # ("YZZYIIIZII", -0.0003518893528389501),
    # ("IIZIIXZZXI", -0.0003518893528389501),
    # ("XZZXIIIZII", -0.0003518893528389501),
    ("IIIZIYZZYI", -0.00901204279263803),
    ("YZZYIIIIZI", -0.00901204279263803),
    ("IIIZIXZZXI", -0.00901204279263803),
    ("XZZXIIIIZI", -0.00901204279263803),
    ("IIIIZYZZYI", 0.0127339139792953),
    ("YZZYIIIIIZ", 0.0107339139792953),
    ("IIIIZXZZXI", 0.0127339139792953),
    ("XZZXIIIIIZ", 0.0127339139792953),
    ("IIIIIYIZYI", -0.003818281201314288),
    ("YIZYIIIIII", -0.003818281201314288),
    ("IIIIIXIZXI", -0.003818281201314288),
    ("XIZXIIIIII", -0.003818281201314288),
    ("IIIIIYZIYI", -0.003818281201314288),
    ("YZIYIIIIII", -0.003818281201314288),
    ("IIIIIXZIXI", -0.003818281201314288),
    ("XZIXIIIIII", -0.003818281201314288),
    ("IYYIIIYYII", 0.004217284878422759),
    ("IXXIIIYYII", 0.004217284878422759),
    ("IYYIIIXXII", 0.004217284878422759),
    ("IXXIIIXXII", 0.004217284878422759),
    # ("IIYZYYZYII", -0.004879740484191498),
    # ("IIXZXYZYII", -0.004879740484191498),
    # ("IIYZYXZXII", -0.004879740484191498),
    # ("IIXZXXZXII", -0.004879740484191498),
    # ("IYZZYYYIII", -0.004879740484191498),
    # ("IXZZXYYIII", -0.004879740484191498),
    # ("IYZZYXXIII", -0.004879740484191498),
    # ("IXZZXXXIII", -0.004879740484191498),
    ("YZYIIYZYII", 0.004868302545087521),
    ("XZXIIYZYII", 0.004868302545087521),
    ("YZYIIXZXII", 0.002868302545087521),
    ("XZXIIXZXII", 0.004868302545087521),
    ("YYIIIYYIII", 0.002868302545087521),
    ("XXIIIYYIII", 0.004868302545087521),
    ("YYIIIXXIII", 0.004868302545087521),
    ("XXIIIXXIII", 0.005868302545087521),
    ("IIIYYIIIII", -0.003917569762418068),
    ("IIIXXIIIII", -0.003917569762418068),
    ("YZZZYIIIII", -0.005949019975734285),
    ("XZZZXIIIII", -0.005949019975734285),
    ("YZZYIIIIII", 0.02355744239583729),
    ("XZZXIIIIII", 0.02355744239583729),
    ("IIIIIIIIII", 1.0709274663656798),
    ("ZIIIIIIIII", -0.5772920990654371),
    ("IZIIIIIIII", -0.4244817531727133),
    ("ZZIIIIIIII", 0.06245512523136934),
    ("IIIIIZZIII", 0.06245512523136934),
    ("ZIZIIIIIII", 0.06245512523136934),
    ("IIIIIZIZII", 0.06245512523136934),
    ("IIZIIIIIII", -0.4244817531727134),
    ("IZZIIIIIII", 0.06558452315458405),
    ("IIIIIIZZII", 0.06558452315458405),
    ("IIIZIIIIII", -0.3899177647415215),
    ("ZIIZIIIIII", 0.053929860773588405),
    ("IIIIIZIIZI", 0.053929860773588405),
    ("IZIZIIIIII", 0.06022550139954594),
    ("IIIIIIZIZI", 0.06022550139954594),
    ("IIZZIIIIII", 0.06022550139954594),
    ("IIIIIIIZZI", 0.06022550139954594),
    ("YZZYZIIIII", 0.006160552555030484),
    ("IIIIIYZZYZ", 0.006460552555030484),
    ("XZZXZIIIII", 0.006160552555030484),
    ("IIIIIXZZXZ", 0.006160552555030484),
    ("IIIIZIIIII", -0.30101532158947975),
    ("IIIIIIIIIZ", -0.30101532158947975),
    ("ZIIIZIIIII", 0.08360121967246183),
    ("IIIIIZIIIZ", 0.08360121967246183),
    ("IZIIZIIIII", 0.06278876343471208),
    ("IIIIIIZIIZ", 0.06278876343471208),
    ("IIZIZIIIII", 0.06278876343471208),
    ("IIIIIIIZIZ", 0.06278876343471208),
    ("IIIZZIIIII", 0.053621410722614865),
    ("IIIIIIIIZZ", 0.053621410722614865),
    ("IIIIIZIIII", -0.5772920990654372),
    ("ZIIIIZIIII", 0.11409163501020725),
    ("IZIIIZIIII", 0.06732342777645686),
    ("ZIIIIIZIII", 0.06732342777645686),
    ("IIZIIZIIII", 0.06732342777645686),
    ("ZIIIIIIZII", 0.06732342777645686),
    ("IIIZIZIIII", 0.060505605672770954),
    ("ZIIIIIIIZI", 0.060505605672770954),
    ("IIIIZZIIII", 0.11433954684977561),
    ("ZIIIIIIIIZ", 0.11433954684977561),
    ("IIIIIIZIII", -0.42448175317271336),
    ("IIIIIIIZII", -0.42448175317271336),
    ("IZIIIIZIII", 0.0782363777898523),
    ("IIZIIIIZII", 0.0782363777898523),
    ("IIZIIIZIII", 0.06980180803300681),
    ("IZIIIIIZII", 0.06980180803300681),
    ("IIIZIIZIII", 0.07055432072184756),
    ("IZIIIIIIZI", 0.07055432072184756),
    ("IIIZIIIZII", 0.07055432072184756),
    ("IIZIIIIIZI", 0.07055432072184756),
    ("IIIIZIZIII", 0.06878552428444665),
    ("IZIIIIIIIZ", 0.06878552428444665),
    ("IIIIZIIZII", 0.06878552428444665),
    ("IIZIIIIIIZ", 0.06878552428444665),
    ("IIIIIIIIZI", -0.38991776474152134),
    ("IIIZIIIIZI", 0.08470391802239534),
    ("IIIIZIIIZI", 0.05665606755281972),
    ("IIIZIIIIIZ", 0.05665606755281972),
    ("IIIIZIIIIZ", 0.12357087224898464),
]


#### optimized_order.py

In [ ]:
import copy
from pprint import pprint as print

unchanged_order = [
    ("IIIYYIIIYY", 0.003034656830204855),
    ("IIIXXIIIYY", 0.003034656830204855),
    ("IIIYYIIIXX", 0.003034656830204855),
    ("IIIXXIIIXX", 0.003034656830204855),
    ("YZZZYIIIYY", -0.008373361424264817),
    ("IIIYYYZZZY", -0.008373361424264817),
    ("XZZZXIIIYY", -0.008373361424264817),
    ("IIIYYXZZZX", -0.008373361424264817),
    ("YZZZYIIIXX", -0.008373361424264817),
    ("IIIXXYZZZY", -0.008373361424264817),
    ("XZZZXIIIXX", -0.008373361424264817),
    ("IIIXXXZZZX", -0.008373361424264817),
    ("YZZYIIIIYY", 0.00211113766859809),
    ("IIIYYYZZYI", 0.00211113766859809),
    ("XZZXIIIIYY", 0.00211113766859809),
    ("IIIYYXZZXI", 0.00211113766859809),
    ("YZZYIIIIXX", 0.00211113766859809),
    ("IIIXXYZZYI", 0.00211113766859809),
    ("XZZXIIIIXX", 0.00211113766859809),
    ("IIIXXXZZXI", 0.00211113766859809),
    ("IIIIIIIIYY", -0.00491756976241806),
    ("IIIIIIIIXX", -0.00491756976241806),
    ("ZIIIIIIIYY", 0.010540187409026488),
    ("IIIYYZIIII", 0.010540187409026488),
    ("ZIIIIIIIXX", 0.010540187409026488),
    ("IIIXXZIIII", 0.010540187409026488),
    ("IZIIIIIIYY", -0.0011822832324725804),
    ("IIIYYIZIII", -0.0011822832324725804),
    ("IZIIIIIIXX", -0.0011822832324725804),
    ("IIIXXIZIII", -0.0011822832324725804),
    ("IIZIIIIIYY", -0.0011822832324725804),
    ("IIIYYIIZII", -0.0011822832324725804),
    ("IIZIIIIIXX", -0.0011822832324725804),
    ("IIIXXIIZII", -0.0011822832324725804),
    ("IIIZIIIIYY", -0.00154067008970742),
    ("IIIYYIIIZI", -0.00154067008970742),
    ("IIIZIIIIXX", -0.00154067008970742),
    ("IIIXXIIIZI", -0.00154067008970742),
    ("IIIIZIIIYY", 0.011733623912074194),
    ("IIIYYIIIIZ", 0.011733623912074194),
    ("IIIIZIIIXX", 0.011733623912074194),
    ("IIIXXIIIIZ", 0.011733623912074194),
    ("IIIIIZIIYY", 0.0027757462269049522),
    ("ZIIYYIIIII", 0.0027757462269049522),
    ("IIIIIZIIXX", 0.0027757462269049522),
    ("ZIIXXIIIII", 0.0027757462269049522),
    ("IIIIIIZIYY", 0.0036202487558837123),
    ("IZIYYIIIII", 0.0036202487558837123),
    ("IIIIIIZIXX", 0.0036202487558837123),
    ("IZIXXIIIII", 0.0036202487558837123),
    ("IIIIIIIZYY", 0.0036202487558837123),
    ("IIZYYIIIII", 0.0036202487558837123),
    ("IIIIIIIZXX", 0.0036202487558837123),
    ("IIZXXIIIII", 0.0036202487558837123),
    ("IIYZYIIYZY", 0.005996760849734561),
    ("IIXZXIIYZY", 0.005996760849734561),
    ("IIYZYIIXZX", 0.005996760849734561),
    ("IIXZXIIXZX", 0.005996760849734561),
    ("IYZZYIYZZY", 0.005996760849734561),
    ("IXZZXIYZZY", 0.005996760849734561),
    ("IYZZYIXZZX", 0.005996760849734561),
    ("IXZZXIXZZX", 0.005996760849734561),
    ("IIYYIIIYZY", 0.004802531988356293),
    ("IIYZYIIYYI", 0.004802531988356293),
    ("IIXXIIIYZY", 0.004802531988356293),
    ("IIYZYIIXXI", 0.004802531988356293),
    ("IIYYIIIXZX", 0.004802531988356293),
    ("IIXZXIIYYI", 0.004802531988356293),
    ("IIXXIIIXZX", 0.004802531988356293),
    ("IIXZXIIXXI", 0.004802531988356293),
    ("IYZYIIYZZY", 0.004802531988356293),
    ("IYZZYIYZYI", 0.004802531988356293),
    ("IXZXIIYZZY", 0.004802531988356293),
    ("IYZZYIXZXI", 0.004802531988356293),
    ("IYZYIIXZZX", 0.004802531988356293),
    ("IXZZXIYZYI", 0.004802531988356293),
    ("IXZXIIXZZX", 0.004802531988356293),
    ("IXZZXIXZXI", 0.004802531988356293),
    ("YZYIIIIYZY", -0.004879740484191497),
    ("XZXIIIIYZY", -0.004879740484191497),
    ("YZYIIIIXZX", -0.004879740484191497),
    ("XZXIIIIXZX", -0.004879740484191497),
    ("YYIIIIYZZY", -0.004879740484191497),
    ("XXIIIIYZZY", -0.004879740484191497),
    ("YYIIIIXZZX", -0.004879740484191497),
    ("XXIIIIXZZX", -0.004879740484191497),
    ("YZZZYYZZZY", 0.0307383271773138),
    ("XZZZXYZZZY", 0.0307383271773138),
    ("YZZZYXZZZX", 0.0307383271773138),
    ("XZZZXXZZZX", 0.0307383271773138),
    ("YZZYIYZZZY", -0.0077644411821215335),
    ("YZZZYYZZYI", -0.0077644411821215335),
    ("XZZXIYZZZY", -0.0077644411821215335),
    ("YZZZYXZZXI", -0.0077644411821215335),
    ("YZZYIXZZZX", -0.0077644411821215335),
    ("XZZZXYZZYI", -0.0077644411821215335),
    ("XZZXIXZZZX", -0.0077644411821215335),
    ("XZZZXXZZXI", -0.0077644411821215335),
    ("IIIIIYZZZY", -0.005949019975734247),
    ("IIIIIXZZZX", -0.005949019975734247),
    ("ZIIIIYZZZY", -0.0351167704024114),
    ("YZZZYZIIII", -0.0351167704024114),
    ("ZIIIIXZZZX", -0.0351167704024114),
    ("XZZZXZIIII", -0.0351167704024114),
    ("IZIIIYZZZY", 0.0027298828353264134),
    ("YZZZYIZIII", 0.0027298828353264134),
    ("IZIIIXZZZX", 0.0027298828353264134),
    ("XZZZXIZIII", 0.0027298828353264134),
    ("IIZIIYZZZY", 0.0027298828353264134),
    ("YZZZYIIZII", 0.0027298828353264134),
    ("IIZIIXZZZX", 0.0027298828353264134),
    ("XZZZXIIZII", 0.0027298828353264134),
    ("IIIZIYZZZY", 0.0023679368995844726),
    ("YZZZYIIIZI", 0.0023679368995844726),
    ("IIIZIXZZZX", 0.0023679368995844726),
    ("XZZZXIIIZI", 0.0023679368995844726),
    ("IIIIZYZZZY", -0.03305872858775587),
    ("YZZZYIIIIZ", -0.03305872858775587),
    ("IIIIZXZZZX", -0.03305872858775587),
    ("XZZZXIIIIZ", -0.03305872858775587),
    ("IIIIIYIZZY", -0.0021498576488650843),
    ("YIZZYIIIII", -0.0021498576488650843),
    ("IIIIIXIZZX", -0.0021498576488650843),
    ("XIZZXIIIII", -0.0021498576488650843),
    ("IIIIIYZIZY", -0.0021498576488650843),
    ("YZIZYIIIII", -0.0021498576488650843),
    ("IIIIIXZIZX", -0.0021498576488650843),
    ("XZIZXIIIII", -0.0021498576488650843),
    ("IIIIIYZZIY", 0.004479074568182561),
    ("YZZIYIIIII", 0.004479074568182561),
    ("IIIIIXZZIX", 0.004479074568182561),
    ("XZZIXIIIII", 0.004479074568182561),
    ("IIYYIIIYYI", 0.010328819322301622),
    ("IIXXIIIYYI", 0.010328819322301622),
    ("IIYYIIIXXI", 0.010328819322301622),
    ("IIXXIIIXXI", 0.010328819322301622),
    ("IYZYIIYZYI", 0.010328819322301622),
    ("IXZXIIYZYI", 0.010328819322301622),
    ("IYZYIIXZXI", 0.010328819322301622),
    ("IXZXIIXZXI", 0.010328819322301622),
    ("YZYIIIIYYI", -0.003466391848475337),
    ("IIYYIYZYII", -0.003466391848475337),
    ("XZXIIIIYYI", -0.003466391848475337),
    ("IIYYIXZXII", -0.003466391848475337),
    ("YZYIIIIXXI", -0.003466391848475337),
    ("IIXXIYZYII", -0.003466391848475337),
    ("XZXIIIIXXI", -0.003466391848475337),
    ("IIXXIXZXII", -0.003466391848475337),
    ("YYIIIIYZYI", -0.003466391848475337),
    ("IYZYIYYIII", -0.003466391848475337),
    ("XXIIIIYZYI", -0.003466391848475337),
    ("IYZYIXXIII", -0.003466391848475337),
    ("YYIIIIXZXI", -0.003466391848475337),
    ("IXZXIYYIII", -0.003466391848475337),
    ("XXIIIIXZXI", -0.003466391848475337),
    ("IXZXIXXIII", -0.003466391848475337),
    ("YZZYIYZZYI", 0.006575744899182541),
    ("XZZXIYZZYI", 0.006575744899182541),
    ("YZZYIXZZXI", 0.006575744899182541),
    ("XZZXIXZZXI", 0.006575744899182541),
    ("IIIIIYZZYI", 0.023557442395837284),
    ("IIIIIXZZXI", 0.023557442395837284),
    ("ZIIIIYZZYI", 0.010889407716094479),
    ("YZZYIZIIII", 0.010889407716094479),
    ("ZIIIIXZZXI", 0.010889407716094479),
    ("XZZXIZIIII", 0.010889407716094479),
    ("IZIIIYZZYI", -0.0003518893528389501),
    ("YZZYIIZIII", -0.0003518893528389501),
    ("IZIIIXZZXI", -0.0003518893528389501),
    ("XZZXIIZIII", -0.0003518893528389501),
    ("IIZIIYZZYI", -0.0003518893528389501),
    ("YZZYIIIZII", -0.0003518893528389501),
    ("IIZIIXZZXI", -0.0003518893528389501),
    ("XZZXIIIZII", -0.0003518893528389501),
    ("IIIZIYZZYI", -0.00901204279263803),
    ("YZZYIIIIZI", -0.00901204279263803),
    ("IIIZIXZZXI", -0.00901204279263803),
    ("XZZXIIIIZI", -0.00901204279263803),
    ("IIIIZYZZYI", 0.0127339139792953),
    ("YZZYIIIIIZ", 0.0127339139792953),
    ("IIIIZXZZXI", 0.0127339139792953),
    ("XZZXIIIIIZ", 0.0127339139792953),
    ("IIIIIYIZYI", -0.003818281201314288),
    ("YIZYIIIIII", -0.003818281201314288),
    ("IIIIIXIZXI", -0.003818281201314288),
    ("XIZXIIIIII", -0.003818281201314288),
    ("IIIIIYZIYI", -0.003818281201314288),
    ("YZIYIIIIII", -0.003818281201314288),
    ("IIIIIXZIXI", -0.003818281201314288),
    ("XZIXIIIIII", -0.003818281201314288),
    ("IYYIIIYYII", 0.004217284878422759),
    ("IXXIIIYYII", 0.004217284878422759),
    ("IYYIIIXXII", 0.004217284878422759),
    ("IXXIIIXXII", 0.004217284878422759),
    ("IIYZYYZYII", -0.004879740484191498),
    ("IIXZXYZYII", -0.004879740484191498),
    ("IIYZYXZXII", -0.004879740484191498),
    ("IIXZXXZXII", -0.004879740484191498),
    ("IYZZYYYIII", -0.004879740484191498),
    ("IXZZXYYIII", -0.004879740484191498),
    ("IYZZYXXIII", -0.004879740484191498),
    ("IXZZXXXIII", -0.004879740484191498),
    ("YZYIIYZYII", 0.004868302545087521),
    ("XZXIIYZYII", 0.004868302545087521),
    ("YZYIIXZXII", 0.004868302545087521),
    ("XZXIIXZXII", 0.004868302545087521),
    ("YYIIIYYIII", 0.004868302545087521),
    ("XXIIIYYIII", 0.004868302545087521),
    ("YYIIIXXIII", 0.004868302545087521),
    ("XXIIIXXIII", 0.004868302545087521),
    ("IIIYYIIIII", -0.004917569762418068),
    ("IIIXXIIIII", -0.004917569762418068),
    ("YZZZYIIIII", -0.005949019975734285),
    ("XZZZXIIIII", -0.005949019975734285),
    ("YZZYIIIIII", 0.02355744239583729),
    ("XZZXIIIIII", 0.02355744239583729),
    ("IIIIIIIIII", 1.0709274663656798),
    ("ZIIIIIIIII", -0.5772920990654371),
    ("IZIIIIIIII", -0.4244817531727133),
    ("ZZIIIIIIII", 0.06245512523136934),
    ("IIIIIZZIII", 0.06245512523136934),
    ("ZIZIIIIIII", 0.06245512523136934),
    ("IIIIIZIZII", 0.06245512523136934),
    ("IIZIIIIIII", -0.4244817531727134),
    ("IZZIIIIIII", 0.06558452315458405),
    ("IIIIIIZZII", 0.06558452315458405),
    ("IIIZIIIIII", -0.3899177647415215),
    ("ZIIZIIIIII", 0.053929860773588405),
    ("IIIIIZIIZI", 0.053929860773588405),
    ("IZIZIIIIII", 0.06022550139954594),
    ("IIIIIIZIZI", 0.06022550139954594),
    ("IIZZIIIIII", 0.06022550139954594),
    ("IIIIIIIZZI", 0.06022550139954594),
    ("YZZYZIIIII", 0.004360552555030484),
    ("IIIIIYZZYZ", 0.004360552555030484),
    ("XZZXZIIIII", 0.004360552555030484),
    ("IIIIIXZZXZ", 0.004360552555030484),
    ("IIIIZIIIII", -0.30101532158947975),
    ("IIIIIIIIIZ", -0.30101532158947975),
    ("ZIIIZIIIII", 0.08360121967246183),
    ("IIIIIZIIIZ", 0.08360121967246183),
    ("IZIIZIIIII", 0.06278876343471208),
    ("IIIIIIZIIZ", 0.06278876343471208),
    ("IIZIZIIIII", 0.06278876343471208),
    ("IIIIIIIZIZ", 0.06278876343471208),
    ("IIIZZIIIII", 0.053621410722614865),
    ("IIIIIIIIZZ", 0.053621410722614865),
    ("IIIIIZIIII", -0.5772920990654372),
    ("ZIIIIZIIII", 0.11409163501020725),
    ("IZIIIZIIII", 0.06732342777645686),
    ("ZIIIIIZIII", 0.06732342777645686),
    ("IIZIIZIIII", 0.06732342777645686),
    ("ZIIIIIIZII", 0.06732342777645686),
    ("IIIZIZIIII", 0.060505605672770954),
    ("ZIIIIIIIZI", 0.060505605672770954),
    ("IIIIZZIIII", 0.11433954684977561),
    ("ZIIIIIIIIZ", 0.11433954684977561),
    ("IIIIIIZIII", -0.42448175317271336),
    ("IIIIIIIZII", -0.42448175317271336),
    ("IZIIIIZIII", 0.0782363777898523),
    ("IIZIIIIZII", 0.0782363777898523),
    ("IIZIIIZIII", 0.06980180803300681),
    ("IZIIIIIZII", 0.06980180803300681),
    ("IIIZIIZIII", 0.07055432072184756),
    ("IZIIIIIIZI", 0.07055432072184756),
    ("IIIZIIIZII", 0.07055432072184756),
    ("IIZIIIIIZI", 0.07055432072184756),
    ("IIIIZIZIII", 0.06878552428444665),
    ("IZIIIIIIIZ", 0.06878552428444665),
    ("IIIIZIIZII", 0.06878552428444665),
    ("IIZIIIIIIZ", 0.06878552428444665),
    ("IIIIIIIIZI", -0.38991776474152134),
    ("IIIZIIIIZI", 0.08470391802239534),
    ("IIIIZIIIZI", 0.05665606755281972),
    ("IIIZIIIIIZ", 0.05665606755281972),
    ("IIIIZIIIIZ", 0.12357087224898464),
]

depth_error_index = [
    (6, -2.5616511021855914e-05, 172),
    (6, -2.5590294317506812e-05, 170),
    (8, -2.4902862737916087e-05, 166),
    (8, -2.488134906016015e-05, 168),
    (10, -2.0592108182990176e-05, 167),
    (6, -2.05643644084208e-05, 173),
    (7, -2.054004309016977e-05, 169),
    (8, -2.0533600628536153e-05, 171),
    (12, -1.796359619714305e-05, 105),
    (9, -1.7559380849768713e-05, 107),
    (10, -1.2228990961912767e-05, 109),
    (7, -1.1739086661508025e-05, 111),
    (0, 6.5627182715966015e-09, 216),
    (6, 9.950046757925546e-06, 114),
    (6, 1.0885222250511806e-05, 112),
    (6, 2.0533984008377915e-05, 115),
    (8, 2.1438116101571425e-05, 113),
    (10, 2.6556020639653677e-05, 104),
    (10, 2.7122596463036164e-05, 106),
    (8, 3.284282635682145e-05, 17),
    (8, 3.703179749942054e-05, 13),
    (11, 3.95847736019278e-05, 19),
    (8, 4.371992295951199e-05, 108),
    (8, 4.379658662663488e-05, 15),
    (8, 4.429556927040834e-05, 110),
    (8, 4.769327549393465e-05, 16),
    (8, 5.2293230802705204e-05, 12),
    (10, 5.426986161749525e-05, 18),
    (10, 5.8889622986668666e-05, 14),
    (10, 7.710035059450837e-05, 152),
    (7, 7.981629695448522e-05, 148),
    (12, 8.522302017774375e-05, 154),
    (8, 9.828620048457637e-05, 144),
    (9, 0.00010070335284595444, 140),
    (10, 0.00010724654546513668, 146),
    (10, 0.00010974122809002951, 142),
    (6, 0.0001159513648057442, 147),
    (8, 0.00011682223517733847, 145),
    (8, 0.00011946542467267118, 143),
    (8, 0.00012027414244279722, 141),
    (0, 0.0001453590584924308, 126),
    (0, 0.00014556020478581622, 124),
    (10, 0.00015092161693422423, 155),
    (10, 0.00015251586906062098, 153),
    (10, 0.0001537249640166244, 151),
    (10, 0.0001552544073140233, 149),
    (2, 0.00015949920827307285, 27),
    (0, 0.00016041290209271863, 122),
    (0, 0.0001605784667576271, 120),
    (2, 0.0001607555374359554, 29),
    (0, 0.00016182600765692057, 184),
    (0, 0.00016531826893552426, 182),
    (2, 0.00016631731201015298, 31),
    (2, 0.0001672269563530865, 33),
    (0, 0.0001681936970073794, 26),
    (0, 0.00016906552710001244, 28),
    (8, 0.00017682856243265987, 35),
    (6, 0.0001771830797241275, 37),
    (0, 0.0001778672244910906, 30),
    (0, 0.00017850335890269575, 32),
    (6, 0.00017969488347473195, 185),
    (7, 0.00018165069764961017, 127),
    (7, 0.00018180502619660754, 125),
    (6, 0.0001841485931034642, 183),
    (7, 0.00018739866326608623, 123),
    (7, 0.0001874665036279144, 121),
    (0, 0.00019143308394631298, 188),
    (6, 0.00019444901826125294, 189),
    (0, 0.00019451892775662616, 186),
    (6, 0.00019813313940526822, 187),
    (17, 0.00019893603817747185, 72),
    (12, 0.00020206936704215972, 70),
    (13, 0.00020491126279301164, 76),
    (12, 0.00020783447740223282, 74),
    (6, 0.00021907484413749567, 34),
    (6, 0.00021936785871402853, 36),
    (12, 0.0002259084293123187, 75),
    (12, 0.00022630310730695213, 77),
    (12, 0.000230746639507437, 71),
    (12, 0.0002309369132276612, 73),
    (10, 0.00024331732266441586, 235),
    (10, 0.00025103552398832163, 233),
    (12, 0.00025764550322884827, 201),
    (15, 0.00026101257595348426, 200),
    (15, 0.0002614363677408693, 199),
    (0, 0.00026254000709652336, 234),
    (12, 0.00026478405665887605, 198),
    (0, 0.00026809102267118246, 236),
    (12, 0.000274508683459157, 85),
    (14, 0.00027800913504721914, 84),
    (14, 0.0002785896664324883, 83),
    (12, 0.00028206265291118304, 82),
    (12, 0.0003028350644034422, 197),
    (14, 0.0003052908301316437, 195),
    (14, 0.0003069357903687786, 196),
    (12, 0.00030937353181989535, 194),
    (12, 0.00031428592201028505, 81),
    (14, 0.0003166054159292886, 80),
    (14, 0.0003190691076118135, 79),
    (12, 0.000321363549265527, 78),
    (10, 0.00032943247995834146, 66),
    (12, 0.0003307595629171989, 68),
    (10, 0.0003328196307620379, 62),
    (12, 0.0003338930816731839, 64),
    (12, 0.0003483216177617987, 54),
    (14, 0.0003490595812428027, 56),
    (14, 0.00034961290113134924, 55),
    (12, 0.00035012700213898906, 57),
    (16, 0.0004332930141961039, 156),
    (19, 0.0004354643938324354, 158),
    (19, 0.0004355624449659534, 157),
    (10, 0.0004355976900998204, 67),
    (10, 0.00043635457708939007, 69),
    (16, 0.0004372661683545165, 159),
    (10, 0.0004400560844465551, 63),
    (10, 0.0004405932385978806, 65),
    (8, 0.0005221058053695993, 206),
    (10, 0.0005226340908446314, 207),
    (10, 0.0005230449585641311, 208),
    (8, 0.000523520005950906, 209),
    (12, 0.0005689588492914094, 202),
    (14, 0.0005694470951559116, 203),
    (14, 0.0005698755890578555, 204),
    (12, 0.0005703710416085223, 205),
    (7, 0.000575326745100968, 131),
    (7, 0.0005758769008213233, 129),
    (16, 0.0005760328949586391, 58),
    (4, 0.0005761511436152, 47),
    (19, 0.000576985963210222, 60),
    (19, 0.0005775580554214371, 59),
    (16, 0.0005783453825060902, 61),
    (4, 0.0005821587948575957, 49),
    (4, 0.0006011427122697977, 51),
    (15, 0.0006056262270389551, 95),
    (4, 0.000606485490712752, 53),
    (15, 0.0006141059401493232, 91),
    (0, 0.0006144800290489721, 46),
    (14, 0.0006181522991100946, 97),
    (0, 0.000619424931846066, 48),
    (0, 0.0006226695868310206, 130),
    (0, 0.0006233849075740344, 128),
    (15, 0.0006263908687529629, 93),
    (0, 0.0006485237379996872, 50),
    (0, 0.0006530436909030768, 52),
    (0, 0.0006612882270047998, 21),
    (0, 0.0006774797082639006, 20),
    (0, 0.000772064163881786, 211),
    (8, 0.0007863436298251314, 3),
    (10, 0.0007869421326732462, 1),
    (10, 0.0007869996909353888, 2),
    (8, 0.0007877263801040063, 0),
    (0, 0.000789691876624185, 210),
    (15, 0.0007931390068276895, 94),
    (15, 0.0008014973965413952, 90),
    (15, 0.0008072429011647397, 96),
    (21, 0.0008155747971838601, 92),
    (4, 0.0008698347161852082, 43),
    (4, 0.0008741342662911405, 45),
    (0, 0.0009088713715612456, 42),
    (0, 0.0009131685826395397, 44),
    (14, 0.001029413791822198, 150),
    (6, 0.0011676999036238783, 174),
    (6, 0.0011729227335822295, 176),
    (10, 0.0012801928801325219, 98),
    (10, 0.0012828404745139355, 99),
    (8, 0.0013920172912457451, 175),
    (10, 0.0013961165185084612, 212),
    (13, 0.0013989762834652752, 213),
    (10, 0.0014034629090225598, 177),
    (10, 0.0015661129771665366, 8),
    (12, 0.0015769044992061387, 10),
    (10, 0.001586254207178464, 4),
    (12, 0.0015965128585331556, 6),
    (10, 0.0017849742693398368, 9),
    (10, 0.0017959165642459224, 11),
    (10, 0.0018101083690956893, 5),
    (10, 0.0018208551026770298, 7),
    (9, 0.002039472653905569, 165),
    (12, 0.0020510830172660866, 163),
    (10, 0.0022138355197533904, 164),
    (13, 0.0022239762964446907, 162),
    (9, 0.0025364919735678254, 179),
    (9, 0.0025565411222830264, 181),
    (23, 0.0026688108546053246, 88),
    (23, 0.002681070729551302, 89),
    (20, 0.002688500183787218, 86),
    (23, 0.002701586159304395, 87),
    (0, 0.00285337831096788, 178),
    (0, 0.002870746675940089, 180),
    (8, 0.0029945657783541635, 190),
    (8, 0.0029945657783543023, 193),
    (10, 0.002994629465211243, 191),
    (10, 0.0029946294652113403, 192),
    (7, 0.0033005528966803394, 41),
    (8, 0.0033022440341942266, 39),
    (6, 0.0037239758565807413, 40),
    (6, 0.0037243899371331157, 38),
    (2, 0.0041889750918018415, 23),
    (2, 0.004196242475382625, 25),
    (0, 0.004399249733022789, 22),
    (0, 0.004406112241215301, 24),
    (10, 0.004600253367998436, 133),
    (8, 0.0046005490311861325, 132),
    (8, 0.0046006490851713405, 135),
    (10, 0.004600974784061171, 134),
    (14, 0.004916798108816484, 137),
    (12, 0.004917315515732573, 139),
    (12, 0.004918097398138838, 136),
    (14, 0.004918710071668622, 138),
    (8, 0.008676318271165911, 215),
    (11, 0.008704726495899859, 214),
    (8, 0.008989030279910479, 161),
    (8, 0.009015394427885376, 160),
    (10, 0.012676973544175046, 119),
    (8, 0.012681071129415772, 117),
    (6, 0.013777817675532891, 118),
    (6, 0.013783813649792642, 116),
    (16, 0.014105415281325737, 100),
    (12, 0.014111891558356793, 102),
    (14, 0.014697721928055824, 101),
    (11, 0.01470237275712516, 103),
    (0, 0.08309958664398885, 245),
    (0, 0.08359147262838558, 246),
    (0, 0.087716761758884, 227),
    (0, 0.08845142755757614, 228),
    (0, 0.09275603015919738, 273),
    (0, 0.09379057305480774, 274),
    (0, 0.09689883503072007, 254),
    (0, 0.09745845060004597, 253),
    (4, 0.09947302485415747, 221),
    (3, 0.09949287326381777, 219),
    (0, 0.09986647225325655, 222),
    (0, 0.09991983756854252, 220),
    (0, 0.10470777465460636, 249),
    (0, 0.10470777508475994, 251),
    (0, 0.10506004785654562, 241),
    (0, 0.10508024682151225, 243),
    (1, 0.10546180212391464, 250),
    (1, 0.10546180249918495, 252),
    (0, 0.10602157240770875, 242),
    (0, 0.10605882576092972, 244),
    (0, 0.11306831230988874, 232),
    (0, 0.11314394367275958, 230),
    (2, 0.1136516447846717, 231),
    (3, 0.11371697470539459, 229),
    (0, 0.11420935661229935, 269),
    (0, 0.11420935808800807, 267),
    (0, 0.11495464969556875, 270),
    (0, 0.11495465018055016, 268),
    (0, 0.1247377486830175, 225),
    (1, 0.12500424274834737, 224),
    (0, 0.13337486443287927, 261),
    (1, 0.13337486448545038, 262),
    (1, 0.13342600357108825, 266),
    (1, 0.13349945422983794, 264),
    (0, 0.1340469720475993, 265),
    (0, 0.13411100080624605, 263),
    (0, 0.15204461552920667, 259),
    (3, 0.15204461569151506, 260),
    (1, 0.15547544563646032, 272),
    (0, 0.1611907126239746, 240),
    (3, 0.16210036532195915, 239),
    (1, 0.17241718055856592, 248),
    (3, 0.19258767195335053, 275),
    (0, 0.22035105628856927, 256),
    (0, 0.22112331404395746, 255),
    (0, 0.5552211269511305, 238),
    (0, 0.555424673750415, 237),
    (0, 0.7452026117442819, 226),
    (1, 0.745557655581386, 271),
    (1, 0.812011164604632, 258),
    (1, 0.8120111646046335, 223),
    (0, 0.8120583976892044, 257),
    (0, 0.8120583976892051, 218),
    (0, 1.0453120685417836, 247),
    (0, 1.0454727160226958, 217),
]


order = copy.deepcopy(unchanged_order)
# for d, e, i in depth_error_index:
#     if e < 0 or d >= 20:
#         order.pop(i)

coeffs = [y for x, y in order]
coeffs = list(set(coeffs))
# print(coeffs)

# error_depth_coeff = sorted(error_depth_coeff, key=lambda x: x[0][0])
# print(error_depth_coeff)
